In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm, Normalize
import os
import cv2 
from PIL import Image 
    
#Import My Library
My_Package_PATH = "/home/boris/Documents/Research/Coding"
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, My_Package_PATH)
#from OneD import *

import OneD.Waves.NonDim as ND
import OneD.NBody.NBody as NB
import OneD.Global.GlobalFuncs as GF

#Set up Directory for saving files/images/videos
# Will not rename this again

Directory = "/home/boris/Documents/Research/Coding/1D codes/Non-Dim"

## Non-Dimensionalized Harmonic Oscillator
Schrodinger Eq.:
$$i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2 \psi}{\partial x^2}+V\psi$$
$V = \frac{1}{2}m\omega^2 x^2$ so:
$$i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m}\frac{\partial^2 \psi}{\partial x^2}+\frac{1}{2}m\omega^2 x^2\psi$$

Fix length scale $L = 1 (\text{kpc or whatever})$, fix velocity scale $v=1 (\text{km/s or whatever})$. 
$$\Rightarrow T = \frac{L}{v} \Rightarrow t\equiv T\tau$$
Also:
$$x\equiv L z$$

Schrodinger Eq. becomes:
$$i \frac{\partial \psi}{\partial \tau} = -\frac{T\hbar}{2mL^2}\frac{\partial^2 \psi}{\partial z^2}+\frac{TV}{\hbar}\psi$$ 

Define
$$r \equiv \frac{T\hbar}{2mL^2} = \frac{\hbar}{2mvL} = \frac{1}{4\pi}\frac{\lambda_\text{deB}}{L}$$

and define a frequency scale $\omega = \frac{2f}{T}$ so:
$$ \frac{TV}{\hbar} = \frac{1}{2}\frac{Tm\omega^2 x^2}{\hbar}=\frac{1}{2}\frac{TmT^{-2}\cdot 4f^2 L^2 z^2}{\hbar} 
= \frac{1}{2}\frac{4mvL}{\hbar}f^2z^2 = \frac{1}{r}f^2z^2$$

The SE becomes:
$$i \frac{\partial \psi}{\partial \tau} = -r\frac{\partial^2 \psi}{\partial z^2}+\frac{1}{r}f^2z^2\psi$$ 


In [2]:
############################################
# Set up shared by Wave and N-Body scenarios
############################################

Num_particles = 100

#Set scales and parameters:
L = 20
v = 1 
hbar = 1
m = 1 #mass per particle
omega = 2*10**(-2)

T = L/v

r = ND.r(hbar,m,v,L)
f = ND.f(omega,v,L)

#Set up Grid
N = 10**3
z = np.linspace(-L/2,L/2,N)
dz = z[1]-z[0]


In [3]:

#Set a fixed potential
def NonDim_phi(z,f):
    return f**2*z**2
phi = NonDim_phi(z,f) 

#Set an initial wavefunction
b=-L/4
std=1
rho = m*GF.gauss(z,b,std)*Num_particles
psi = np.sqrt(rho/m) #assuming psi_initial to be real valued and non-negative

#Calculate initial Density perturbation
rho_avg = np.mean(rho)
P = rho-rho_avg

#storage space, with initial functions
P_s = np.array([P]) #initial density perturbation
psi_s = np.array([psi]) #initial wavefunction
phi_s = np.array([phi]) #initial potential [perturbation]

dtau = 0.1
tau_stop = 20
time = 0
while time <= tau_stop:
    ######################################
    #Evolve system forward by one time-step
    psi,phi,P = ND.time_evolve_FixedPhi(psi,phi,r,dz,dtau,m)
    ######################################

    #Append/store new arrays
    psi_s = np.append(psi_s,[psi], axis = 0)
    P_s = np.append(P_s, [P], axis = 0)
    phi_s = np.append(phi_s,[phi], axis = 0)

    time += dtau #forward on the clock

In [ ]:
#Plot the figures
folder_name = "HO_Images"
GF.plot_save_waves(z,psi_s,phi_s,P_s,dtau,Num_particles,Directory,folder_name)
print("Plotting Done.")

In [ ]:
#Turn figures into a video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_name = "ToyModel_HarmonicOscillator_video.mp4"
GF.animate(fourcc,Directory,folder_name,video_name,dtau)


In [ ]:
# Open the video
! xdg-open "ToyModel_HarmonicOscillator_video.mp4"

In [ ]:
#Testing phase representation
from matplotlib.colors import Normalize

k = 2*np.pi*np.fft.fftfreq(len(z),dz)
k = k / L #non-dimensionalize
dk = k[1]-k[0]

#rescale wavenumber k to velocity v:
v = k*(hbar/m)

x_min, x_max = np.min(z), np.max(z)
v_min, v_max = np.min(v), np.max(v)

for i in range(0,200,25):
    F = ND.Husimi_phase(psi_s[i],z,dz,L)
    print("Normalization Check: ",np.sum(dz*dk*F))
    #fig,ax = plt.subplots(1,1)
    plt.figure(figsize = (10,5))
    plt.imshow(F,extent = (x_min,x_max,v_min,v_max),cmap = cm.hot, aspect = x_max/v_max)
    plt.xlim([x_min,x_max])
    plt.ylim([v_min,v_max])
    plt.colorbar()
    plt.show()

In [ ]:
max_F = 2
frame_spacing = 1
folder_name = "HO_Phase_Images"
ND.generate_phase_plots(psi_s,z,dz,L,m,hbar,max_F,frame_spacing,Directory,folder_name)

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_name = "HO_Phase_video.mp4"
GF.animate(fourcc,Directory,folder_name,video_name,dtau)


In [ ]:

! xdg-open "HO_Phase_video.mp4"

### Non-dimensionalized Kick-Drift-Kick

In the general non-dimensionalized case:
$$i\frac{\partial \psi}{\partial \tau} = -r \frac{\partial ^2 \psi}{\partial z^2} + \frac{Tm\Phi}{\hbar}\psi$$
Kick:
$$\exp\left( {-i\frac{Lm}{v\hbar}\frac{\Delta \tau}{2}\Phi(z,\tau+\Delta \tau)} \right)$$

Drift:
$$ \mathcal{F^{-1}} \exp\left( -ir \Delta \tau k^2 \right)\mathcal{F}$$

## N-body Harmonic Oscillator
#### Non-dimensionalized

Considering the above Length and Velocity scales $L$,$v$, as well as parameters $r$ and $f$, the SHO equation of motion (for a point particle) becomes:
$$
\frac{d^2 z}{d \tau^2} = -4f^2z
$$

In [4]:
Num_stars = Num_particles
m = 1 #mass per star
M = m*Num_stars #total mass

#T = L/v
r_ = ND.r(hbar,m,v,L)
f_ = ND.f(omega,v,L)

#Set a fixed potential
def NonDim_phi(z,m,omega):
    return 0.5*m*omega**2*z**2
phi = NonDim_phi(z,m,omega) 
#calculate the acceleration
#g = -np.gradient(phi,dz)
def g(z):
    return -4* f_**2 * z

#Set initial distribution on grid
b = -L/4 #center at zero
std = 1 #standard deviation of 1
z_0 = np.random.normal(b,std,Num_stars) #initial positions sampled from normal distribution
stars = [NB.star(i,m,z_0[i],0) for i in range(len(z_0))] #create list of normally distributed stars, zero initial speed

folder_name = "HO_NBody_Images"
os.chdir(Directory + "/" + folder_name)

dtau = 0.1
t_stop = 20
time = 0
i = 0 #counter, for saving images
while time <= t_stop:
    fig,ax = plt.subplots(1,3)
    fig.set_size_inches(30,10)
    plt.suptitle(f"Time {round(dtau*i,1)}")
    
    ax[0].plot(z,phi,label = "Potential")

    x_s = np.array([])
    v_s = np.array([])

    for my_star in stars:
        #1. PLOT FIRST
        x = my_star.x
        v = my_star.v
        x_s = np.append(x_s,x)
        v_s = np.append(v_s,v)

        ax[0].plot(x,0,".")
        ax[0].set_xlim([-L/2,L/2])
        #ax[0].set_ylim([np.min(positions),np.max(positions)])
        ax[0].legend()

        ax[1].plot(x,v,".")
        ax[1].set_xlim([-L/2,L/2])
        ax[1].set_ylim([-4,4])
        #ax[1].legend()

        #2. THEN EVOLVE SYSTEM
        my_star.evolve_star_dynamics(g,dtau)

    heat = ax[2].hist2d(x_s,v_s,bins = [100,100],range = [[-L/2,L/2],[-2,2]],cmap = cm.hot)
    #ax[2].set_colorbar()
    ax[2].set_xlim(-L/2,L/2)
    ax[2].set_ylim(-2,2)
    #plt.colorbar(heat, ax[2])

    #now save it as a .jpg file:
    folder = Directory + "/" + folder_name
    filename = 'ToyModelPlot' + str(i+1).zfill(4) + '.jpg';
    plt.savefig(folder + "/" + filename)  #save this figure (includes both subplots)
    
    plt.close() #close plot so it doesn't overlap with the next one
    
    time += dtau
    i += 1

In [5]:
folder_name = "HO_NBody_Images"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_name = "HO_NBody_video.mp4"
GF.animate(fourcc,Directory,folder_name,video_name,dtau)


In [6]:

! xdg-open "HO_NBody_video.mp4"